# Práctica 2: Aprendizaje y selección de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

### Autores:

* Marina Ceballos Verdejo
* Jesús Martínez Garrido

Cargamos las librerías necesarias

In [ ]:
# Third party
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.metrics import  precision_score, recall_score, roc_auc_score

import time

# Local application
import utilidad_p2 as utils
import utilidad as utils_p1

Fijamos una semilla:

In [ ]:
random_state = 27912

# 1. Selección de modelos: Diabetes

## 1.1 Carga de datos

Cargamos el conjunto de datos diabetes:

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

index_col = None
target = "Outcome"

data = utils.load_data(filepath, index_col, target)

Comprobando que se ha cargado correctamente:

In [ ]:
data.sample(5, random_state=random_state)

Dividimos el conjunto de datos en las variables predictoras (X) y la variable clase (y):

In [ ]:
target = "Outcome"

(X, y) = utils.divide_dataset(data, target)

Comprobamos que se han separado correctamente:

In [ ]:
X.sample(5, random_state=random_state)

In [ ]:
y.sample(5, random_state=random_state)


Por último, dividimos el conjunto de datos en entrenamiento y prueba mediante un *holdout* estratificado:

In [ ]:
stratify = y
train_size = 0.75

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)

Comprobamos:

In [ ]:
X_train.sample(5, random_state=random_state)

In [ ]:
y_train.sample(5, random_state=random_state)

In [ ]:
X_test.sample(5, random_state=random_state)

In [ ]:
y_test.sample(5, random_state=random_state)

### **X e y ahora serán nuestros datos de entrenamiento:**

In [ ]:
X=X_train
y=y_train

## 1.2 Creación de los modelos

### 1.2.1 Vecinos más cercanos

In [ ]:
n_neighbors = 10

k_neighbors_model = KNeighborsClassifier(n_neighbors)

Aquí creamos el modelo de KNN (en este caso k es 10). Hay que tener en cuenta que si elegimos un número bastante alto de vecinos el modelo tenderá a parecerse a 0R ya que al final se obtiene la mayoría de los vecinos más cercanos (así funciona 0R pero saca la clase mayoritaria de todo el conjunto de entrenamiento). También debemos de tener en cuenta que si creamos un clasificador con una k más pequeña nuestro modelo tenderá a sobreajustarse. Por lo que debemos de elegir bien este parámetro k para que nuestro modelo no tenga mucho sesgo ni varianza. 

### 1.2.2 Árboles de decisión

In [ ]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)

In [ ]:
model = decision_tree_model

utils.plot_tree(model, X, y)

Aquí podemos ver como se ha creado un árbol de decisión completo (sin poda). Se puede apreciar que la mayoría de las hojas tiene un número de ejemplos menor a 3 por lo que podríamos decir que este árbol tiene bastante sobreajuste. 

In [ ]:
max_depth = 3

pre_decision_tree_model = DecisionTreeClassifier(max_depth=max_depth,
                                                 random_state=random_state)

In [ ]:
model = pre_decision_tree_model

utils.plot_tree(model, X, y)

Aquí, a diferencia del anterior, obtenemos un árbol con poco sobreajuste ya que tenemos hojas con una gran cantidad de ejemplos (menos la segunda hoja que tiene solo 4 ejemplos , que podría tener sobreajuste). También debemos de tener en cuenta que el sesgo ha amuentado con respecto al anterior árbol ya que tenemos un árbol mucho menos complejo (con muchas menos hojas).

O post-poda:

In [ ]:
ccp_alpha = 0.005

post_decision_tree_model = DecisionTreeClassifier( ccp_alpha=ccp_alpha,
                                                  random_state=random_state)

In [ ]:
model = post_decision_tree_model

utils.plot_tree(model, X, y)

Aquí obtenemos un modelo con poca varianza (como en el caso anterior) pero que puede haber aumentando el sesgo.

### 1.2.3 Adaptative Boosting (AdaBoost)

In [ ]:
adaboost_model = AdaBoostClassifier(random_state=random_state)

Ahora creamos un modelo adaboost con árboles de clasificación que tienen como nivel de profundidad máxima 1, 50 estimadores en total y tasa de aprendizaje 1.

### 1.2.4 Bootstrap Aggregating (Bagging)

In [ ]:
bagging_model = BaggingClassifier(random_state=random_state)

Creamos un modelo con los hiperparámetros por defecto. Es decir, se usarán 10 árboles de clasificación como estimadores. 

### 1.2.5 Random Forests

In [ ]:
random_forest_model = RandomForestClassifier(random_state=random_state)

Creamos un random forest con 100 árboles que usan el criterio gini y max_features 'sqrt'.

### 1.2.6 Gradient Tree Boosting (Gradient Boosting)

In [ ]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

En este clasificador se usarán 100 árboles como estimadores. Con una funcion de pérdida "deviance" y criterio usado "friedman_mse".

### 1.2.7 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

In [ ]:
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)

Creamos el modelo Histogram-Based Gradient Boosting con una función de pérdida "auto".

## 1.3 Evaluación de modelos

Para ello vamos a utilizar validación cruzada iterada:

In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

Creamos los transformadores que vamos a pasar a la pipeline (práctica 1):

In [ ]:
nan_put=utils_p1.ManualNanPut({'Glucose':[50,250], 'BloodPressure':[10,150], 'Pregnancies':[0,50], 'SkinThickness':[1,80], 'Insulin':[0,650], 'BMI':[10,65], 'DiabetesPedigreeFunction':[0.0,3.0], 'Age':[0,100], })#anómalos y ruidosos a nan
imp = SimpleImputer(missing_values=float('nan'), strategy='mean')#valores perdidos
feature_selector=utils_p1.ManualFeatureSelector([0,1,2,4,5,6,7])#Imputación variable SkinThickness
discretizer = KBinsDiscretizer(n_bins=2, strategy="kmeans")#Discretizar

Para la creación de pipelines, vamos a usar discretizado en todos los modelos en los que se creen árboles ya que en la anterior práctica obteniamos un mejor árbol cuando discretizabamos. En el Histogram Gradient Boosting no se usará el discretizado ya que el propoio modelo discretiza los datos.

Evaluamos nuestros modelos usando el área bajo la curva:

In [ ]:
model=make_pipeline(nan_put, imp, feature_selector, discretizer, decision_tree_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put, imp, feature_selector, discretizer, pre_decision_tree_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put, imp, feature_selector, discretizer, post_decision_tree_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put, imp, feature_selector, k_neighbors_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put, imp, feature_selector, discretizer, adaboost_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put, imp, feature_selector, discretizer, bagging_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put, imp, feature_selector, discretizer, random_forest_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put, imp, feature_selector, discretizer, gradient_boosting_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put, imp, feature_selector, hist_gradient_boosting_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

Notamos que los árboles con poda tienen un mejor resultado en el test que el de sin poda. Esto es porque el árbol sin poda está sobreajustado, lo que hace que se cometan muchos más errores que en un árbol con poda (aunque en los árboles con poda encontramos también errores que podrían ser causados por el sesgo ya que son demasiado pequeños). También notamos que en el árbol sin poda el score usando los datos de entrenamiento es mayor, esto ocurre porque el árbol es tan complejo que se ha ramificado totalmente con los datos de entrenamiento (el sobreajuste en este caso no afecta).
    
El mejor clasificador en este caso es Adaboost. Sabemos que este clasificador normalmente funciona bastante bien (en muchos casos supera a bagging), también debemos de tener en cuenta que este clasificador tarda bastante tiempo en entrenar (a diferencia de los demás), lo que puede ser un punto en contra. Lo mismo pasa con otros ensembles, ofrecen buenos resultados pero tardan más tiempo en entrenar que los demás.


# 1.4 Selección de modelos

Al igual que en la práctica anterior, vamos a utilizar tres métricas distintas, estas son: el área bajo la curva, recall y precisión.

In [ ]:
scoring={'AUC': 'roc_auc', 'Recall':'recall', 'Precision':'precision'}

A la hora de seleccionar la configuración óptima, vamos a hacer una media de las tres métricas anteriores, este valor quedará reflejado en la tabla en la columna "mean_scores".

### 1.4.1 Vecinos más cercanos

In [ ]:
estimator = k_neighbors_model
pipe = Pipeline(steps=[('nan_put', nan_put),('imp', imp),('feature_selector', feature_selector), ('knn', estimator)])




weights = ["uniform", "distance"]
n_neighbors = [2, 3, 5, 8, 10, 12, 15, 20, 25, 30, 35]
metric = ["euclidean", "manhattan", "minkowski"]

k_neighbors_clf = utils.optimize_params(pipe,
                                        X, y, cv,
                                        scoring=scoring,
                                        knn__weights=weights,
                                        knn__n_neighbors=n_neighbors,
                                        knn__metric=metric)

Para la selección de hiperparámetros hemos usado tres:
* n_neighbors: es el hiperparámetro imprescindible para este estimador (se trata del número de vecinos más cercanos que se va a usar en el algoritmo). Hemos decidido usar números altos de vecinos ya que números menores generarían demasiado sobreajuste en nuestro modelo (y con la cantidad de datos que disponemos, pensamos que escoger menos vecinos no sería acertado). Además, como vimos en los diagramas de dispersión de la práctica 1, nuestros datos se encuentran bastantes dispersos (es decir, los puntos de ambas clases se encontraban bastante superpuestos) por lo que no se debería coger un número bajo de vecinos. También, hemos puesto 35 como el número máximo de vecinos ya que más de esto haría que nuestro problema aumentase el sesgo. Hemos ido saltando algunos valores para que la validación cruzada no se sobrecargue y tarde demasiado tiempo. 
* weights: hiperparámetro usado para decidir los pesos de los vecinos más cercanos. Conocemos dos formas de decidir estos pesos: uniform (pesos iguales) y distance(pesos asignados en función a la inversa de la distancia).
* metric: es utilizado para calcular las distancias entre los puntos. Conocemos tres: la euclidea (en línea recta), manhattan (como diferencia de las coordenadas de los puntos) y minkowski (generalización de ambas).

Como solución hemos obtenido la siguiente:

{'metric': 'manhattan', 'knn__n_neighbors': 15, 'knn__weights': 'uniform'}
 

* Obtenemos que el número de vecinos que se va a utilizar es 15. Es un número que concuerda con nuestro problema ya que, como hemos dicho anteriormente, nuestros puntos se encuentran bastante dispersos (con esto impedimos que nuestro modelo se sobreajuste). El número tampoco es demasiado grande para evitar aumentar el sesgo de nuestro modelo.
* También vemos que funciona mejor con el hiperparámetro "weights" como "uniform" y "metric" como "manhattan".


### 1.4.2 Árbol de decisión

In [ ]:
# Should not modify the original model
estimator = clone(decision_tree_model)
pipe = Pipeline(steps=[('nan_put', nan_put),('imp', imp),('feature_selector', feature_selector), ('discretizer', discretizer),('tree', estimator)])

criterion = ["gini", "entropy"]
max_depth = [3, 5, 7, 10, 20, 25]
min_samples_leaf= [5, 10, 20, 40]
ccp_alpha = [0, 0.001, 0.0025,0.005]

decision_tree_clf = utils.optimize_params(pipe,
                                        X, y, cv,
                                        scoring=scoring,
                                        tree__criterion = criterion,
                                        tree__max_depth = max_depth,
                                        tree__min_samples_leaf= min_samples_leaf,
                                        tree__ccp_alpha = ccp_alpha)

Para la selección de hiperparámetros hemos usado cuatro:
* criterion: se trata del criterio para seleccionar los umbrales usados en cada nodo del árbol.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Sabiendo la cantidad de ejemplos que tiene nuestro conjunto, hemos puesto que el mínimo sea 3 porque pensamos que un árbol con una profundidad menor tendría mucho sesgo (ya que, como vimos en la práctica uno, sabemos que nuestros datos están bastante dispersos) y con una profundidad mayor a 25 tendría mucho sobreajuste.
* min_samples_leaf: es un hiperparámetro que nos indica el número mínimo de ejemplos que tendrá cada hoja del árbol (prepoda). Hemos puesto que 5 sea el número mínimo ya que menos ejemplos haría que el árbol tuviera sobreajuste y 40 como máximo ya que un número mayor aumentaría el sesgo.
* ccp_alpha: parámetro usado para la postpoda. Viendo nuestro árbol dibujado anteriormente con un ccp de 0.005 hemos decidido poner los valores en torno a este porque hemos encontrado un árbol de tamaño medio para el número de ejemplos que tenemos.

En este caso nos encontramos nuevamente con varios empates:
* {'tree__ccp_alpha': 0.005, 'tree__criterion': 'entropy', 'tree__max_depth': 10, 'tree__min_samples_leaf': 20}
* {'tree__ccp_alpha': 0.005, 'tree__criterion': 'entropy', 'tree__max_depth': 25, 'tree__min_samples_leaf': 20}
* {'tree__ccp_alpha': 0.005, 'tree__criterion': 'entropy', 'tree__max_depth': 20, 'tree__min_samples_leaf': 20}
* {'tree__ccp_alpha': 0.005, 'tree__criterion': 'entropy', 'tree__max_depth': 5, 'tree__min_samples_leaf': 20}
* {'tree__ccp_alpha': 0.005, 'tree__criterion': 'entropy', 'tree__max_depth': 7, 'tree__min_samples_leaf': 20}

El hiperparámetro max_depth hace que ocurran estos empates. Esto es porque la postpoda y la prepoda con min_samples_leaf hace que se nos quede un árbol con una profundidad igual o menor a 5, lo que hace que si el hiperparámetro max_depth es mayor o igual a 5 el árbol no cambie nada.
Las demás configuraciones de hiperparámetros son:
* ccp_alpha: Con un valor de 0.005. Ya que sabemos que nuestro "mejor" árbol tiene una profundidad igual o menor a cinco, este hiperparámetro nos ayuda a reducir más la varianza del árbol, por ello, es mayor a 0.
* Para el hiperparámetro "criterion" vemos que el mejor valor es la entropía.
* Por último, el mejor valor para el hiperparámetro min_samples_leaf es 20. Nos parece que 20 como min_samples_leaf es correcto para equilibrar el error debido a sesgo y varianza. Un valor mayor haría que el sesgo aumentara y un valor menor haría que la varianza aumentara.

Hemos obtenido un árbol que se encuentra bastante podado (usando pre y post-poda), esto es, como hemos dicho antes, para combatir la varianza que generaría un árbol demasiado complejo.

### 1.4.3 Adaptative Boosting (AdaBoost)

In [ ]:
estimator = adaboost_model
pipe= Pipeline(steps=[('nan_put', nan_put),('imp', imp),('feature_selector', feature_selector), ('discretizer', discretizer),('adaboost', estimator)])



# Should not modify the base original model
tree_estimator = clone(decision_tree_model)

base_estimator = [tree_estimator]
learning_rate = [0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.015]
n_estimators = [20, 50, 80]
n_bins=[2,3]

adaboost_clf = utils.optimize_params(pipe,
                                     X, y, cv,
                                     scoring=scoring,
                                     adaboost__base_estimator=base_estimator,
                                     adaboost__learning_rate=learning_rate,
                                     adaboost__n_estimators=n_estimators,
                                     adaboost__base_estimator__criterion=criterion,
                                     adaboost__base_estimator__max_depth=max_depth,
                                     adaboost__base_estimator__ccp_alpha=ccp_alpha,
                                     discretizer__n_bins=n_bins)


Para la selección de hiperparámetros hemos usado cinco (aparte del base_estimator):
* learning_rate: reduce la contribución de cada clasificador.
* criterion: se trata del criterio para seleccionar los umbrales usados en cada nodo del árbol.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Sabiendo que adaboost funciona con modelos simples hemos puesto valores bajos en la profundidad máxima para obtener árboles con mucho sesgo. Este sesgo será compensado por el conjunto de todos los árboles.
* ccp_alpha: parámetro usado para la postpoda. Hemos decidido poner dos valores, uno en el que no existe postpoda (0) y otro en el que existe algo de poda (0.015), hemos aumentado este valor al del ejemplo ya que son árboles demasiado pequeños.
* n_estimators: es el número de modelos que se van a aplicar en el adaboost. Por defecto son 50 estimadores, hemos puesto dos valores en torno a 50 para ver si hay mejores resultados.
* n_bins: es el número de bins usado en la discretización. En la práctica 1 explicamos que usar 2 bins era una buena opción para el árbol, queremos comprobar si 3 puede ser una buena opción en este modelo.

Para la selección de mejores hiperparámetros se ha obtenido tres empates:
* {'base_estimator': DecisionTreeClassifier(ccp_alpha=0.015, criterion='entropy', max_depth=1,random_state=27912), 'ccp_alpha': 0.015, 'criterion': 'entropy', 'max_depth': 1, 'learning_rate': 0.95, 'n_estimators': 20, 'n_bins': 3}
* {'base_estimator': DecisionTreeClassifier(ccp_alpha=0.015, criterion='entropy', max_depth=1,random_state=27912), 'ccp_alpha': 0.015, 'criterion': 'entropy', 'max_depth': 1, 'learning_rate': 0.95, 'n_estimators': 50, 'n_bins': 3}
* {'base_estimator': DecisionTreeClassifier(ccp_alpha=0.015, criterion='entropy', max_depth=1,random_state=27912), 'ccp_alpha': 0.015, 'criterion': 'entropy', 'max_depth': 1, 'learning_rate': 0.95, 'n_estimators': 80, 'n_bins': 3}

Parece curioso, que en este caso se comporte de la misma forma un clasificador con 20 estimadores que con 50 y 80. Pensamos que esto es así porque con 20 estimadores (para esta combinación de hiperparámetros) se obtienen buenas predicciones, si añadimos más estimadores, únicamentes estaríamos añadiendo complejidad al problema porque la asignación de pesos estaría haciendo que siempre se generaran las mismas predicciones.

Podemos ver que nuestros árboles son muy simples (porque tienen máxima profundidad 1), esto es porque Adaboost funciona con modelos simples (el algoritmo completo de Adaboost es capaz de corregir el sesgo de cada árbol individual al usar pesos en cada ejemplo). 
También, nuestros árboles usan post-poda, algo que los hace aún más simples.
Nos damos cuenta, que en la mayoría de configuraciones, Adaboost funciona mejor cuando el número de intervalos por los que discretizamos es 3.

Por último, cabe destacar que hemos obtenido un empate de tres configuraciones, pero con la que nos deberíamos de quedar es la que tiene el número de estimadores igual a 20, ya que al tener menos estimadores consume menos tiempo en el entrenamiento y en la predicción.

### 1.4.4 Bootstrap Aggregating (Bagging)

In [ ]:
estimator = bagging_model
pipe= Pipeline(steps=[('nan_put', nan_put),('imp', imp),('feature_selector', feature_selector), ('discretizer', discretizer),('bagging', estimator)])

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
max_depth = [2, 3, 4, 5]
ccp_alpha = [0.0, 0.015]
n_estimators = [5, 10, 20]
n_bins=[2,3]

bagging_clf = utils.optimize_params(pipe,
                                    X, y, cv,
                                    scoring=scoring,
                                    bagging__base_estimator=base_estimator,
                                    bagging__n_estimators=n_estimators,
                                    bagging__base_estimator__criterion=criterion,
                                    bagging__base_estimator__max_depth=max_depth,
                                    bagging__base_estimator__ccp_alpha=ccp_alpha,
                                    discretizer__n_bins=n_bins)

Para la selección de hiperparámetros hemos usado cinco (aparte del base_estimator):
* criterion: se trata del criterio para seleccionar los umbrales usados en cada nodo del árbol.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Sabiendo que Bagging funciona con modelos más complejos que Adaboost (en este caso se buscan árboles que tengan bastante varianza para reducirse con el total de ellos) hemos puesto valores más altos en la profundidad máxima.
* ccp_alpha: parámetro usado para la postpoda. Hemos decidido poner dos valores, uno en el que no existe postpoda (0) y otro en el que existe algo de poda (0.015), hemos aumentado este valor al del ejemplo ya que son árboles demasiado pequeños.
* n_estimators: es el número de modelos que se van a aplicar en el adaboost. Por defecto son 10 estimadores, hemos puesto dos valores en torno a 10 para ver si hay mejores resultados.
* n_bins: al igual que en el anterior caso, queremos comprobar con qué número de bins sería mejor discretizar para este clasificador.

En este caso, solo encontramos una configuración óptima:
{'base_estimator': DecisionTreeClassifier(ccp_alpha=0.015, criterion='entropy', max_depth=4, random_state=27912), 'ccp_alpha': 0.015, 'criterion': 'entropy', 'max_depth': 4, 'bagging__n_estimators': 20, 'n_bins':3}


Obtenemos árboles con una profundidad máxima 4, que usan post-poda y el criterio entropy. Estos árboles son más complejos que los generados por Adaboost, esto es porque Bagging funciona con estimadores más complejos (corrige le error debido a la varianza con la suma de todos los árboles).

Se ha usado el mayor número de estimadores de entre los posibles. En teoría, los ensembles cuanto más estimadores, mejor funcionan, vemos que en este caso esto se cumple.

Por último, cabe destacar que el número de intervalos para la discretización que mejor funciona es 3. Pensamos que esto es debido a la simplicidad de los árboles en ambos estimadores (Bagging y Adaboost), esto añade algo más de información a los árboles.



### 1.4.5 Random Forests

In [ ]:
estimator = random_forest_model
pipe= Pipeline(steps=[('nan_put', nan_put),('imp', imp),('feature_selector', feature_selector), ('discretizer', discretizer),('forest', estimator)])

criterion = ["gini", "entropy"]
max_features = ["sqrt"]
n_estimators = [50, 100, 150]
max_depth= [1, 2, 3, 4, 5]
ccp_alpha=[0.0, 0.015]
n_bins=[2,3]

random_forest_clf = utils.optimize_params(pipe,
                                          X, y, cv,
                                          scoring=scoring,
                                          forest__criterion=criterion,
                                          forest__max_features=max_features,
                                          forest__n_estimators=n_estimators,
                                          forest__max_depth=max_depth,
                                          forest__ccp_alpha=ccp_alpha,
                                          discretizer__n_bins=n_bins)

Para la selección de hiperparámetros hemos usado cinco:
* criterion: se trata del criterio para seleccionar los umbrales usados en cada nodo del árbol.
* max_features: es el número máximo de características que se van a usar en cada árbol. Se puede usar "sqrt" que hace la raíz cuadrada del número total de variables y "log2" que hace el logaritmo en base 2, pero, como tenemos 7 variables predictoras (y una imputada) logaritmo en base 2 de 7 y raíz cuadrada de 7 obtienen resultados parecidos (redondeado es 3), por lo que decidimos solo usar una de ellas.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Sabemos que Random Forest funciona con modelos más complejos que Adaboost (con más varianza) pero el algoritmo compensa esta varianza con la suma de todos los árboles.
* ccp_alpha: parámetro usado para la postpoda. Hemos decidido poner dos valores, uno en el que no existe postpoda (0) y otro en el que existe algo de poda (0.015), hemos aumentado este valor al del ejemplo ya que son árboles demasiado pequeños.
* n_estimators: es el número de modelos que se van a aplicar en el adaboost. Por defecto son 100 estimadores, hemos puesto dos valores en torno a 100 para ver si hay mejores resultados.
* n_bins: igual que en los apartados anteriores

Los valores obtenidos son:
{'ccp_alpha': 0.0, 'criterion': 'entropy', 'max_depth': 4, 'max_features': 'sqrt','n_estimators': 150, 'n_bins':3}

En este caso hemos obtenido árboles más complejos que los dos ensembles anteriores (se diferencia del anterior en que no se aplica post-poda). Esto es así porque Random Forest usa modelos más complejos que Adaboost, pero que es capaz de compensar el error debido a la varianza con la suma de todos los árboles.
También, podemos ver que se sigue cumpliendo la teoría de que cuantos más estimadores mejor funciona el ensemble (ya que se ha seleccionado el número mayor de estimadores).

Por último, cabe destacar que la tercera configuración de hiperparámetros también es una buena opción ya que no se diferencia mucho de la primera y utiliza menos tiempo en el entrenamiento y la predicción (ya que usa un número menor de estimadores).

### 1.4.6 Gradient Tree Boosting (Gradient Boosting)

In [ ]:
estimator = gradient_boosting_model
pipe= Pipeline(steps=[('nan_put', nan_put),('imp', imp),('feature_selector', feature_selector), ('discretizer', discretizer),('gBoosting', estimator)])

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3, 4]
ccp_alpha = [0.0, 0.015]
n_estimators = [50, 100, 150]

gradient_boosting_clf = utils.optimize_params(pipe,
                                              X, y, cv,
                                              scoring=scoring,
                                              gBoosting__learning_rate=learning_rate,
                                              gBoosting__criterion=criterion,
                                              gBoosting__max_depth=max_depth,
                                              gBoosting__ccp_alpha=ccp_alpha,
                                              gBoosting__n_estimators=n_estimators)

Para la selección de hiperparámetros hemos usado cinco:
* learning_rate: es la contribución de cada árbol.
* criterion: se trata del criterio para seleccionar partición.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Sabemos que Gradient Boosting funciona con modelos simples por lo que hemos puesto valores bajos en la profundidad máxima para obtener árboles con mucho sesgo. Este sesgo será compensado por el conjunto de todos los árboles.
* ccp_alpha: parámetro usado para la postpoda. Hemos decidido poner dos valores, uno en el que no existe postpoda (0) y otro en el que existe algo de poda (0.015), hemos aumentado este valor al del ejemplo ya que son árboles demasiado pequeños.
* n_estimators: es el número de modelos que se van a aplicar en el adaboost. Por defecto son 100 estimadores, hemos puesto dos valores en torno a 100 para ver si hay mejores resultados.

Hemos obtenido la siguiente configuración:
{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 50}

En este caso se han seleccionado árboles algo más complejos (comparando con AdaBoost) ya que tienen un nivel de profundidad más y no tienen post-poda. Aún así, son árboles simples por lo que también funciona bastante bien.

En este caso, el número de estimadores es el más bajo. Según hemos dicho antes, en teoría cuantos más estimadores mejor, pero en la práctica no todo es así y aquí lo podemos comprobar. Pensamos que se han seleccionado menos porque tenemos árboles algo más complejos por lo que más estimadores podrían hacer que el error debido a varianza aumentara.

Como tenemos menos estimadores, esta configuración tarda menos en entrenar y predecir que otras que usan más estimadores.

### 1.4.7 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

In [ ]:
estimator = hist_gradient_boosting_model
pipe= Pipeline(steps=[('nan_put', nan_put),('imp', imp),('feature_selector', feature_selector),('histGBoosting', estimator)])


learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]
max_depth = [1,3,5,7,10,15]

hist_gradient_boosting_clf = utils.optimize_params(pipe,
                                                   X, y, cv,
                                                   scoring=scoring,
                                                   histGBoosting__learning_rate=learning_rate,
                                                   histGBoosting__max_leaf_nodes=max_leaf_nodes,
                                                   histGBoosting__max_depth=max_depth)

Para la selección de hiperparámetros hemos usado tres:
* learning_rate: es la contribución de cada árbol.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Ya que "Histogram Gradient Boosting" funciona con modelos simples hemos decidido poner valores bajos en la profundidad máxima para obtener árboles con mucho sesgo. Este sesgo se irá compensandoo con la suma de todos los árboles.
* max_leaf_nodes: es el número máximo de nodos hoja de cada árbol(prepoda). En este caso (a diferencia de los demás) hay un número predeterminado de max_leaf_nodes (31) por tanto decidimos poner valores en torno a 31.

En el último modelo de ellos obtenemos la configuración óptima:
{'learning_rate': 0.04, 'max_depth': 3, 'max_leaf_nodes': 15}

En este algoritmo, cabe destacar que se han creado árboles más complejos que el anterior. Pensamos que esto es así porque en este caso se puede discretizar usando más de dos intervalos (a diferencia del anterior) lo que añade complejidad al problema.

## 1.5 Construcción y validación del modelo final

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf
}

In [ ]:
X = X_test
y = y_test

utils.evaluate_estimators(estimators, X, y)

De acuerdo con estos resultados, podemos concluir que los ensembles se comportan mejor que knn y el árbol de decisión (es algo que suele pasar). Aunque, el Gradient Boosting funciona bastante peor que los demás ensembles, pensamos que esto es porque se han usado árboles demasiado complejos (con profundidad 2 y sin post-poda) para este algoritmo, lo que hace que se cometa un sobreajuste, este modelo queda descartado ya que nos brinda malos resultados y es de los que más tardan en entrenar. Se puede notar que AdaBoost tiene árboles más simples y es de los modelos que mejor resultados nos brinda.

Cabe destacar que el árbol de decisión y knn nos brindan resultados que son bastante buenos y tardan bastante poco en entrenar y predecir, por lo que podrían ser una buena solución en caso de que se busquen unos modelos rápidos.

Por último, los dos modelos que mejores resultados obtienen son AdaBoost y Histogram Gradient Boosting destancando entre ellos el segundo. Su tuvieramos que elegir uno sería el Histogram Gradient Boosting ya que los dos tardan más o menos el mismo tiempo en entrenar y predecir y este obtiene unos mejores resultados.

# 2. Selección de modelos: Wisconsin

## 2.1 Carga de datos

Cargamos el conjunto de datos Wisconsin:

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index_col = None
target = "diagnosis"

data = utils.load_data(filepath, index_col, target)

Comprobando que se ha cargado correctamente:

In [ ]:
data.sample(5, random_state=random_state)

Dividimos el conjunto de datos en las variables predictoras (X) y la variable clase (y)

In [ ]:
target = "diagnosis"

(X, y) = utils.divide_dataset(data, target)

Eliminamos la columna Unnamed:

In [ ]:
del X['Unnamed: 32']

Comprobamos que se han separado correctamente:

In [ ]:
X.sample(5, random_state=random_state)

In [ ]:
y.sample(5, random_state=random_state)

Por último, dividimos el conjunto de datos en entrenamiento y prueba mediante un holdout estratificado:

In [ ]:
stratify = y
train_size = 0.75

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)

Comprobamos:

In [ ]:
X_train.sample(5, random_state=random_state)

In [ ]:
y_train.sample(5, random_state=random_state)

In [ ]:
X_test.sample(5, random_state=random_state)

In [ ]:
y_test.sample(5, random_state=random_state)

### **X e y ahora serán nuestros datos de entrenamiento:**

In [ ]:
X=X_train
y=y_train

## 2.2 Creación de los modelos

Creamos modelos similares al apartado de la base de datos anterior.

### 2.2.1 Vecinos más cercanos

In [ ]:
n_neighbors = 10

k_neighbors_model = KNeighborsClassifier(n_neighbors)

### 2.2.2 Árboles de decisión

In [ ]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)

In [ ]:
model = decision_tree_model

utils.plot_tree(model, X, y)

Pre-poda:

In [ ]:
max_depth = 3

pre_decision_tree_model = DecisionTreeClassifier(max_depth=max_depth,
                                                 random_state=random_state)

In [ ]:
model = pre_decision_tree_model

utils.plot_tree(model, X, y)

Post-poda

In [ ]:
ccp_alpha = 0.005

post_decision_tree_model = DecisionTreeClassifier( ccp_alpha=ccp_alpha,
                                                  random_state=random_state)

In [ ]:
model = post_decision_tree_model

utils.plot_tree(model, X, y)

### 2.2.3 Adaptative Boosting (AdaBoost)

In [ ]:
adaboost_model = AdaBoostClassifier(random_state=random_state)

### 2.2.4 Bootstrap Aggregating (Bagging)

In [ ]:
bagging_model = BaggingClassifier(random_state=random_state)

### 2.2.5 Random Forests

In [ ]:
random_forest_model = RandomForestClassifier(random_state=random_state)

### 2.2.6 Gradient Tree Boosting (Gradient Boosting)

In [ ]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

### 2.2.7 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

In [ ]:
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)

## 2.3 Evaluación de modelos

Para ello vamos a utilizar validación cruzada iterada:

In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

Creamos los transformadores usados en la pipeline de la práctica anterior:

In [ ]:
nan_put_anomalos=utils_p1.ManualNanPut({'radius_mean':[6.981,22.27], 'texture_mean':[10.38,29.43], 'perimeter_mean':[43.79,153.5], 'area_mean':[143.5,1386], 'smoothness_mean':[0.06251, 0.1326], 'compactness_mean':[0.01938, 0.2276], 'concavity_mean':[0,0.2871], 'concave points_mean':[0,0.152],'symmetry_mean':[0.1203, 0.2459], 'fractal_dimension_mean':[0.04996, 0.0787], 'radius_worst':[7.93,28.4], 'texture_worst':[12.49,40.68], 'perimeter_worst':[50.41,188.5], 'area_worst':[185.2,2089], 'smoothness_worst':[0.08125, 0.1862], 'compactness_worst':[0.02729, 0.6247], 'concavity_worst':[0,0.7681], 'concave points_worst':[0,0.291],'symmetry_worst':[0.1506, 0.4154], 'fractal_dimension_worst':[0.05521, 0.1224] })
nan_put_ruidosos=utils_p1.ManualNanPutSE({'radius':1.2, 'texture':2.5, 'perimeter':18.5, 'area':229.99, 'smoothness':0.015, 'compactness':0.03, 'concavity':0.025, 'concave points':0.015,'symmetry':0.036, 'fractal_dimension':0.0085 })
imp = SimpleImputer(missing_values=float('nan'), strategy='mean')
feature_selector=utils_p1.ManualFeatureSelector([0, 1, 4, 8, 9, 20, 21, 23, 24, 28, 29])
discretizer = KBinsDiscretizer(n_bins=5, strategy="uniform")

Evaluamos nuestros modelos:

In [ ]:
model=make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, discretizer, decision_tree_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, discretizer, pre_decision_tree_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, discretizer, post_decision_tree_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, discretizer, k_neighbors_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, discretizer, adaboost_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, discretizer, bagging_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, discretizer, random_forest_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, discretizer, gradient_boosting_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

In [ ]:
model=make_pipeline(nan_put_anomalos, nan_put_ruidosos, imp, feature_selector, hist_gradient_boosting_model)

utils.evaluate_estimator(model, X, y, cv, scoring='roc_auc')

Aquí pasa lo que en la base de datos anterior. El árbol sin poda obtiene peores resultados que los que tienen poda debido a la varianza.
Vemos que los ensembles y el knn son los que mejores datos nos brindan, destacando entre ellos el Histogram Gradient Boosting. También, nos damos cuenta que el knn es el que menos tiempo tarda en entrenar por lo que también sería un modelo bastante bueno (obtiene una buena relación entre el score y tiempo de ejecución).

## 2.4 Selección de modelos

Al igual que la base de datos anterior, usaremos las métricas: área bajo la curva, recall y precision.

In [ ]:
scoring={ 'AUC':'roc_auc', 'Recall':make_scorer(recall_score, pos_label="M"), 'Precision':make_scorer(precision_score, pos_label="M")}

### 2.4.1 Vecinos más cercanos

In [ ]:
estimator = k_neighbors_model
pipe = Pipeline(steps=[('nan_put_anomalos', nan_put_anomalos),('nan_put_ruidosos', nan_put_ruidosos),('imp', imp),('feature_selector', feature_selector),('knn', estimator)])


weights = ["uniform", "distance"]
n_neighbors = [2, 3, 5, 7, 10, 15, 20]
metric = ["euclidean", "manhattan", "minkowski"]

k_neighbors_clf = utils.optimize_params(pipe,
                                        X, y, cv,
                                        scoring=scoring,
                                        knn__weights=weights,
                                        knn__n_neighbors=n_neighbors,
                                        knn__metric=metric)

Para la selección de hiperparámetros hemos usado tres:
* n_neighbors: es el hiperparámetro imprescindible para este estimador (se trata del número de vecinos más cercanos que se va a usar en el algoritmo). En este caso hemos puesto números más bajos de vecinos que en la anterior base de datos. Esto es porque en nuestro diagrama de dispersión (práctica 1), nuestros puntos no se encuentran tan superpuestos los unos de los otros (es decir, las fronteras son más claras), por lo que, en este caso no hay tanto peligro de sobreajuste.
* weights: hiperparámetro usado para decidir los pesos de los vecinos más cercanos. Conocemos dos formas de decidir estos pesos: uniform (pesos iguales) y distance(pesos asignados en función a la inversa de la distancia).
* metric: es utilizado para calcular las distancias entre los puntos. Conocemos tres: la euclidea (en línea recta), manhattan (como diferencia de las coordenadas de los puntos) y minkowski (generización de ambas).

Como solución hemos obtenido tres resultados empates:

{metric=manhattan, n_neighbors=3, weights=uniform}.

* Obtenemos que el número de vecinos que se va a utilizar es 3. Como hemos dicho antes, las fronteras están bastante claras, lo que hace que un número bajo de vecinos se comporte bastante bien (no hay tanto peligro de sobreajuste con números bajos).
* También vemos que el hiperparámetro "weights" toma el valor "uniform" y el hiperparámetro "metric" el valor "manhattan". Por lo que, el algoritmo se comporta mejor asignando el mismo peso a los tres vecinos más cercanos usando la distancia de manhattan.

### 2.4.2 Árbol de decisión

In [ ]:
# Should not modify the original model
estimator = clone(decision_tree_model)
pipe = Pipeline(steps=[('nan_put_anomalos', nan_put_anomalos),('nan_put_ruidosos', nan_put_ruidosos),('imp', imp),('feature_selector', feature_selector),('discretizer', discretizer),('tree', estimator)])

criterion = ["gini", "entropy"]
max_depth = [3, 5, 10, 20]
min_samples_leaf= [5, 10, 20, 40]
ccp_alpha = [0, 0.0025,0.005]

decision_tree_clf = utils.optimize_params(pipe,
                                        X, y, cv,
                                        scoring=scoring,
                                        tree__criterion = criterion,
                                        tree__max_depth = max_depth,
                                        tree__min_samples_leaf= min_samples_leaf,
                                        tree__ccp_alpha = ccp_alpha)

Para la selección de hiperparámetros hemos usado cuatro:
* criterion: se trata del criterio para seleccionar los umbrales usados en cada nodo del árbol.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Sabiendo la cantidad de ejemplos que tiene nuestro conjunto, hemos puesto que el mínimo sea 3 porque pensamos que un árbol con una profundidad menor tendría mucho sesgo (ya que, como vimos en la práctica uno, sabemos que nuestros datos están bastante dispersos) y con una profundidad mayor a 25 tendría mucho sobreajuste. También, como las fronteras están algo más claras que en la base de datos anterior, no necesitamos un árbol tan profundo
* min_samples_leaf: es un hiperparámetro que nos indica el número mínimo de ejemplos que tendrá cada hoja del árbol (prepoda). Hemos puesto que 5 sea el número mínimo ya que menos ejemplos haría que el árbol tuviera sobreajuste y 40 como máximo ya que un número mayor aumentaría el sesgo.
* ccp_alpha: parámetro usado para la postpoda. Viendo nuestro árbol dibujado anteriormente con un ccp de 0.005 hemos decidido poner los valores en torno a este porque hemos encontrado un árbol de tamaño medio para el número de ejemplos que tenemos.

En este caso nos encontramos nuevamente con tres empates:
* {'ccp_alpha': 0, 'criterion': 'gini', 'tree__max_depth': 10, 'tree__min_samples_leaf': 5}
* {'ccp_alpha': 0, 'criterion': 'gini', 'tree__max_depth': 25, 'tree__min_samples_leaf': 5}
* {'ccp_alpha': 0, 'criterion': 'gini', 'tree__max_depth': 20, 'tree__min_samples_leaf': 5}

En este caso el max_depth es el valor que hace que ocurran estos empates. Esto es porque el árbol resultado al usar gini, prepoda (con min_samples_leaf igual a 5) y sin postpoda tiene una profundidad igual o menor a 10, lo que hace que si el hiperparámetro max_depth es mayor o igual a 10 el árbol no cambie nada.
Las demás configuraciones de hiperparámetros son:
* ccp_alpha: Con un valor de 0. Sabemos que este árbol tiene una profundidad menor o igual a 10, el número mínimo de ejemplos igual a 5. Ya que no se ha aplicado la post-poda podemos deducir que si se aplicara aumentaría el error de nuestro árbol por sesgo.
* Para el hiperparámetro "criterion" vemos que el mejor valor es gini.
* Por último, el mejor valor para el hiperparámetro min_samples_leaf es 5. Sabiendo el tamaño de nuestro conjunto, nos parece que 5 es un número correcto para equilibrar sesgo y varianza dentro del árbol (un mayor valor haría que el árbol aumentara el sesgo y un valor menor aumentaría la varianza).
También, vemos que las configuraciones que se encuentran en las siguientes posiciones tardan más o menos el mismo tiempo en entrenar y predecir, por lo que lo acertado sería quedarse con la primera configuración.

### 2.4.3 Adaptative Boosting (AdaBoost)

In [ ]:
estimator = adaboost_model
pipe = Pipeline(steps=[('nan_put_anomalos', nan_put_anomalos),('nan_put_ruidosos', nan_put_ruidosos),('imp', imp),('feature_selector', feature_selector),('discretizer', discretizer),('adaboost', estimator)])



# Should not modify the base original model
tree_estimator = clone(decision_tree_model)

base_estimator = [tree_estimator]
learning_rate = [0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.015]
n_estimators = [20, 50, 80]


adaboost_clf = utils.optimize_params(pipe,
                                     X, y, cv,
                                     scoring=scoring,
                                     adaboost__base_estimator=base_estimator,
                                     adaboost__learning_rate=learning_rate,
                                     adaboost__n_estimators=n_estimators,
                                     adaboost__base_estimator__criterion=criterion,
                                     adaboost__base_estimator__max_depth=max_depth,
                                     adaboost__base_estimator__ccp_alpha=ccp_alpha)


Para la selección de hiperparámetros hemos usado cinco (aparte del base_estimator):
* learning_rate: reduce la contribución de cada clasificador.
* criterion: se trata del criterio para seleccionar los umbrales usados en cada nodo del árbol.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Como sabemos que adaboost funciona con modelos simples hemos puesto valores bajos en la profundidad máxima y así obtener árboles con mucho sesgo. Este sesgo será compensado por la suma de todos los árboles.
* ccp_alpha: parámetro usado para la postpoda. Hemos decidido poner dos valores, uno en el que no existe postpoda (0) y otro en el que existe algo de poda (0.015), se ha aumentado este valor al del ejemplo ya que son árboles demasiado pequeños.
* n_estimators: es el número de modelos que se van a aplicar en el adaboost. Por defecto son 50 estimadores, hemos puesto dos valores en torno a 50 para ver si hay mejores resultados.


Para la selección de mejores hiperparámetros se ha obtenido dos empates:
* {'base_estimator': DecisionTreeClassifier(max_depth=1, random_state=27912), 'ccp_alpha': 0.0, 'criterion': 'entropy', 'max_depth': 1, 'learning_rate': 1.0, 'n_estimators': 20}


Podemos ver que nuestros árboles son muy simples (porque tienen máxima profundidad 1), esto es porque Adaboost funciona con modelos simples (el algoritmo completo de Adaboost es capaz de corregir el sesgo de cada árbol individual). 
También, nuestros árboles no usan postpoda (el ccp_alpha es 0) porque nuestros árboles son muy simples (ya se ha hecho prepoda).
El número de estimadores que usa nuestro problema es 20. En teoría, este algoritmo funciona mejor cuantos más estimadores, pero aquí vemos un ejemplo práctico de que esto no es así.

También, nos damos cuenta que al tener únicamente 20 estimadores, este algoritmo tarda menos en entrenar que los que tienen más estimadores. Por lo que, aquí encontramos un algoritmo con un rendimiento bastante bueno.

### 2.4.4 Bootstrap Aggregating (Bagging)

In [ ]:
estimator = bagging_model
pipe = Pipeline(steps=[('nan_put_anomalos', nan_put_anomalos),('nan_put_ruidosos', nan_put_ruidosos),('imp', imp),('feature_selector', feature_selector),('discretizer', discretizer),('bagging', estimator)])

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
max_depth = [2, 3, 4]
ccp_alpha = [0.0, 0.015]
n_estimators = [5, 10, 20]
n_bins=[4,5]

bagging_clf = utils.optimize_params(pipe,
                                    X, y, cv,
                                    scoring=scoring,
                                    bagging__base_estimator=base_estimator,
                                    bagging__n_estimators=n_estimators,
                                    bagging__base_estimator__criterion=criterion,
                                    bagging__base_estimator__max_depth=max_depth,
                                    bagging__base_estimator__ccp_alpha=ccp_alpha,
                                    discretizer__n_bins=n_bins)

Para la selección de hiperparámetros hemos usado cinco (aparte del base_estimator):
* criterion: se trata del criterio para seleccionar los umbrales usados en cada nodo del árbol.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Como Bagging funciona con estimadores más complejos que Adaboost hemos aumentado en este caso la profundidad máxima.
* ccp_alpha: parámetro usado para la postpoda. Se ha puesto dos valores, uno en el que no existe postpoda (0) y otro en el que existe algo de poda (0.015), se ha aumentado este valor al del ejemplo ya que son árboles demasiado pequeños.
* n_estimators: es el número de modelos que se van a aplicar en el adaboost. Por defecto son 10 estimadores, hemos puesto dos valores en torno a 10 para ver si hay mejores resultados.
* n_bins: como vimos en la anterior práctica, lo mejor era discretizar el árbol con 5 bins. Queremos ver si discretizamos con 4 bins obtenemos mejores resultados.

En este caso, solo encontramos una configuración óptima:

{'base_estimator': DecisionTreeClassifier(max_depth=4, random_state=27912), 'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 4, 'bagging__n_estimators': 20, 'n_bins': 5}

Como hemos dicho antes, Bagging funciona con estimadores más complejos, por lo que hemos obtenido árboles de profundidad máxima 4, sin post-poda y que usan el criterio gini. El error debido a varianza (ya que son árboles complejos) se corrige con el total de todos ellos.

Normalmente, los ensembles funcionan mejor cuantos más estimadores tengan. Por eso se ha seleccionado el número mayor del hiperparámetro "n_estimators".

También, podemos observar que la segunda y tercera configuración consumen menos tiempo de entrenamiento y predicción (ya que usan menos estimadores), por lo que, podrían ser soluciones potenciales. 

### 2.4.5 Random Forests

In [ ]:
estimator = random_forest_model
pipe = Pipeline(steps=[('nan_put_anomalos', nan_put_anomalos),('nan_put_ruidosos', nan_put_ruidosos),('imp', imp),('feature_selector', feature_selector),('discretizer', discretizer),('forest', estimator)])

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
n_estimators = [50, 100]
max_depth= [2, 3, 4]
ccp_alpha=[0.0, 0.015]

random_forest_clf = utils.optimize_params(pipe,
                                          X, y, cv,
                                          scoring=scoring,
                                          forest__criterion=criterion,
                                          forest__max_features=max_features,
                                          forest__n_estimators=n_estimators,
                                          forest__max_depth=max_depth,
                                          forest__ccp_alpha=ccp_alpha)

Para la selección de hiperparámetros hemos usado cinco:
* criterion: se trata del criterio para seleccionar los umbrales usados en cada nodo del árbol.
* max_features: es el número máximo de características que se van a usar en cada árbol. Se puede usar "sqrt" que hace la raíz cuadrada del número total de variables y "log2" que hace el logaritmo en base 2.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda).Al igual que Bagging, Random Forest funciona con modelos algo más complejos (con más varianza) que AdaBoost por ello se han decidido poner valores más altos.
* ccp_alpha: parámetro usado para la postpoda. Hemos decidido poner dos valores, uno en el que no existe postpoda (0) y otro en el que existe algo de poda (0.015), hemos aumentado este valor al del ejemplo ya que son árboles más pequeños.
* n_estimators: es el número de modelos que se van a aplicar en el adaboost. Por defecto son 100 estimadores, hemos puesto dos valores en torno a 100 para ver si hay mejores resultados.

Hemos obtenido la siguiente configuración óptima:
{'ccp_alpha': 0.0, 'criterion': 'entropy', 'max_depth': 4,'max_features': 'sqrt','n_estimators': 50}

En este apartado obtenemos árboles equivalentes al apartado anterior. Esto es porque Random Forest funciona de forma equivalente a Boosting, es decir, se buscan modelos más complejos que en AdaBoost.

También, observamos que el número de estimadores que se usa es el menor de entre las posibilidades (de hecho encontramos en la posición 3 la misma configuración pero con 100 estimadores). Por lo tanto, podemos deducir que, para esta combinación, si aumentamos los estimadores aumentamos el error debido a varianza.

Cabe destacar que esta configuración obtiene buenos números en cuanto al tiempo consumido añadiendo más valor a esta solución.

### 2.4.6 Gradient Tree Boosting (Gradient Boosting)

In [ ]:
estimator = gradient_boosting_model
pipe = Pipeline(steps=[('nan_put_anomalos', nan_put_anomalos),('nan_put_ruidosos', nan_put_ruidosos),('imp', imp),('feature_selector', feature_selector),('discretizer', discretizer),('gBoosting', estimator)])

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.015]
n_estimators = [50, 100]

gradient_boosting_clf = utils.optimize_params(pipe,
                                              X, y, cv,
                                              scoring=scoring,
                                              gBoosting__learning_rate=learning_rate,
                                              gBoosting__criterion=criterion,
                                              gBoosting__max_depth=max_depth,
                                              gBoosting__ccp_alpha=ccp_alpha,
                                              gBoosting__n_estimators=n_estimators)

Para la selección de hiperparámetros hemos usado cinco:
* learning_rate: es la contribución de cada árbol.
* criterion: se trata del criterio para seleccionar partición.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (prepoda). Al igual que en AdaBoost, Gradient Boosting funciona con estimadores simples por lo que se han puesto números bajos.
* ccp_alpha: parámetro usado para la postpoda. Se han puesto los mismos valores que en apartados anteriores
* n_estimators: es el número de modelos que se van a aplicar en el adaboost. Por defecto son 100 estimadores, hemos puesto dos valores en torno a 100 para ver si hay mejores resultados.

En este caso encontramos un empate de configuraciones:
* {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
* {'ccp_alpha': 0.0, 'criterion': 'mse', 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}

Las dos configuraciones que han empatado se diferencian en el criterio que mide la calidad de la división. "friedman_mse" usa una puntuación de mejora en el MSE. De estos resultados podemos deducir que, para esta combinación de hiperparámetros, la función de mejora de Friedman no aporta valor al algoritmo.

Como se podía esperar, hemos obtenido árboles bastantes simples ya que, a diferencia de los dos anteriores, Gradient Boosting funciona con estimadores simples.

### 2.4.7 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

In [ ]:
estimator = hist_gradient_boosting_model
pipe = Pipeline(steps=[('nan_put_anomalos', nan_put_anomalos),('nan_put_ruidosos', nan_put_ruidosos),('imp', imp),('feature_selector', feature_selector),('histGBoosting', estimator)])


learning_rate = [ 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]
max_depth = [1,3,5,10,15]

hist_gradient_boosting_clf = utils.optimize_params(pipe,
                                                   X, y, cv,
                                                   scoring=scoring,
                                                   histGBoosting__learning_rate=learning_rate,
                                                   histGBoosting__max_leaf_nodes=max_leaf_nodes,
                                                   histGBoosting__max_depth=max_depth)

Para la selección de hiperparámetros hemos usado tres:
* learning_rate: es la contribución de cada árbol.
* max_depth: es la profundidad máxima que puede alcanzar nuestro árbol (pre-poda). Sabemos que "Histogram Gradient Boosting" funciona con modelos simples por lo que hemos puesto valores bajos en la profundidad máxima para obtener árboles con mucho sesgo. Este sesgo será compensado por el conjunto de todos los árboles.
* max_leaf_nodes: es el número máximo de nodos hoja de cada árbol(pre-poda). En este caso (a diferencia de los demás) hay un número predeterminado de max_leaf_nodes (31) por tanto decidimos poner valores en torno a 31.

Hemos obtenido la siguiente configuración óptima:
{'learning_rate': 0.05, 'max_depth': 5, 'max_leaf_nodes': 15}

En este caso obtenemos árboles algo más complejos que Gradient Boosting y AdaBoost. Pensamos que esto es así porque este algoritmo discretiza las variables continuas en un número mayor de bins y esto hace que aumente la complejidad de nuestros datos (necesitamos árboles más complejos).


## 2.5 Construcción y validación del modelo final

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf
}

In [ ]:
X = X_test
y = y_test

utils.evaluate_estimators(estimators, X, y, pos_label='M')

Viendo estos datos, el modelo que peor resultados brinda es el de Nearest neighbors. Pensamos que esto es así porque tenemos bastantes variables y KNN se comporta de forma que todas las variables se consideran de igual importancia, lo que hace que una variable que aporte poco al problema le quite importancia a una variable que sea significativa.

El árbol de decisión obtiene muy buenos resultados, ya que, a diferencia del KNN, este algoritmo es capaz de darle más importancia a algunas variables.

Como era de esperar, los ensembles también obtienen buenos resultados. AdaBoost es de los que tienen resultados más bajos, esto podría ser por el error debido a sesgo, ya que sus estimadores son demasiado simples (aunque AdaBoost trabaja con estimadores simples) y únicamente tiene 20. Random Forests también obtiene resultados bajos (comparado con los demás ensembles) pensamos que también es porque se usan pocos estimadores para reducir la varianza que generan los árboles. Bagging obtiene resultados mejores que los dos anteriores, se han usado árboles complejos para reducir la varianza con el total de todos ellos.

Por último, Gradient Boosting e Histogram Gradient Boosting obtienen muy buenos resultados. En Gradient Boosting usa árboles algo más complejos que AdaBoost, lo que ha hecho que se obtengan mejores resultados. El que mejor se comporta de todos ellos es el Histogram Gradient Boosting ya que este algoritmo aporta al anterior una discretización de variables.

Si tenemos en cuenta el tiempo de entrenamiento y predicción, Gradient Boosting, Bagging y Decision Tree son unos modelos que pueden ser bastante buenos ya que obtienen buenos resultados y su tiempo de ejecución es significativamente más bajo que Histogram Gradient Boosting.


# 3. Kernel seleccionado

Nuestro kernel seleccionado es el que se encuentra en el siguiente enlace:
[Kernel](http://www.kaggle.com/youhanlee/would-it-be-possible-to-predict-success-of-app)

Este kernel trata de crear un modelo que sea capaz de predecir la valoración de los usuarios de aplicaciones del app store. En concreto se va a predecir si una aplicación es buena (1) o mala (0). Usa una base de datos que está disponible en kaggle en el siguiente enlace:
[Dataset](http://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

Con esta base de datos se puede predecir la valoración media del usuario ([0,5]) pero el creador del kernel ha decidido discretizar estos valores de modo que se pueda decidir si una aplicación es buena (cuando la valoración media es mayor a 4) o mala (valoración menor a 4). La base de datos está compuesta por dos conjuntos, un conjunto con información básica de la app (como precio, valoración, tamaño) y otro con la descripción (nombre y descripción de la app).

A continuación se va a describir el proceso que se sigue en el kernel:

Primero carga los datos, comprueba valores nulos, fusiona los dos conjuntos y muestra los primeros 5 ejemplos (debería de haber utilizado samples en vez del head ya que con los cinco primeros ejemplos no se puede sacar información que represente toda la base de datos).

A continuación, añade dos nuevas variables al conjunto. Una de ellas es el tamaño de la app en MB pero pensamos que esto no es necesario porque las dos variables de tamaño van a estar correlacionadas (nos aportan la misma información las dos). Otra de ellas es una variable categórica que indica si la aplicación des gratis o no (usando la variable precio), esta variable pensamos que sí que puede aportar valor al problema.

Por lo tanto, de momento tenemos de variables predictoras:
* "id".
* "track_name": Nombre de la app.
* "size_bytes": Tamaño en Bytes.
* "size_bytes_in_MB": Tamaño en MB.
* "price".
* "isNotFree".
* "rating_count_tot": Cantidad de valoraciones de usuarios teniendo en cuenta todas las versiones de la aplicación.
* "rating_count_ver": Cantidad de valoraciones de usuarios de la última versión de la app.
* "user_rating_ver": Valoración media del usuario de la última versión de la app.
* "ver" : Código de la última versión
* "cont_rating".
* "prime_genre".
* "sup_devices.num": número de dispositivos soportados por la app.
* "ipadSc_urls.num": número de capturas de pantalla mostradas.
* "lang.num": número de idiomas.
* "vpp_lic": Licencia vpp.
* "app_desc": Descripción de la app.

Y como variable clase:
* "user_rating" : Valoración media del usuario teniendo en cuenta todas las versiones de la aplicación.

A continuación, vemos como se ha incluido un análisis exploratorio en el kernel y podemos sacar las siguientes conclusiones:
* Los juegos son las aplicaciones que más aparecen en esta base de datos (con bastante diferencia).
* Las aplicaciones de educación, productividad y salud suelen ser de pago y los juegos, aplicaciones de entretenimiento y redes sociales suelen ser gratis.
* Los géneros mejor valorados son "Productivity", "Music" y "Photo & Video" y los que peor "Book", "Finance" y "Catalogs"
* Los géneros "Book", "Catalogs" y "Navigation" tienen una mejor valoración cuando son de pago.
* Los géneros "Medical" y "Games" son los que más memoria utilizan.
* Las aplicaciones medicas, juegos y libros suelen pesar más si son de pago
* Las aplicaciones más costosas son del género "Medical", "Music", "Catalogs" y "Business" y las menos costosas son "social network", "news", "finance" y "shopping".
* Del diagrama de correlación podemos ver que las variables que ha creado el autor del kernel están correlacionadas (es algo que suponíamos). También vemos que "user_rating" y "user_rating_ver" están algo correlacionadas (directamente) con "ipadSc_urls.num" y "lang.num" por lo que estas dos últimas variables nos podrían aportar bastante valor al problema.
* Las palabras más frecuentes en descripciones de aplicaciones con buena valoración son: "game", "app", "the", "http", "new".

Luego realiza dos preprocesamientos (uno sin la descripción y otro con la descripción):
* Añade la variable "rating_count_before" que es el número de valoraciones totales sin contar la versión actual.
* Convierte variables categóricas a indicadores.
* Genera la variable predictora (1 si el rating es mayor a 4 y 0 si es menor o igual).
* Genera la partición en train y test.

Para la parte que tiene la descripción añade dos variables distintas, una con el tamaño de la descripción y otra categórica (1 si aparece "game" y 0 si no)

En todo este proceso hemos encontrado un error. Debería de haber generado la partición train y test al principio del kernel y haber hecho el análisis exploratorio con el conjunto train. En el preprocesamiento debería de haber creado una pipeline donde aplicara todas las transformaciones y no hacerlo sobre todo el conjunto directamente.

Para la selección de modelos realiza una validación cruzada de 5 divisiones, para ello utiliza tres clasificadores distintos:
* Random Forest.
* LGBMClassifier de la librería lightgbm: es el equivalente al Gradient Boosting.
* XGBClassifier de xgboost: también equivale al Gradient Boosting.

Utiliza los hiperparámetros por defecto por lo tanto tenemos:
* Random Forest: 100 estimadores, criterio gini, sin pre-poda ni post-poda, con la raíz cuadrada de número de características para cada árbol.
* lightgbm: tasa de aprendizaje 0.1, 100 estimadores, se aplica una pre-poda del número máximo de hojas (31) y número mínimo de ejemplos por hoja (20).
* XGBClassifier: tasa de aprendizaje 0.3, máxima profundidad para los árboles de 6.

Después de realizar una validación cruzada con los datos de entrenamiento obtiene que el mejor modelo es el XGBClassifier (en el conjunto que no tiene descripción y el que sí la tiene).

Usa el resultado de la validación cruzada como la evaluación final (no ha usado el conjunto de test). Esto es un error, porque el resultado de la validación cruzada debería de utilizarse para seleccionar el modelo correcto y después obtener la evaluación final usando el conjunto de test. 
